# مومنت

تابع مومنت سه نوع مومنت مکانی ومرکزی و مرکزی نرمال را با استفاده از فرمول گرین محاسبه میکند

# moments = cv2.moments(arr)  or  dst = cv2.HuMoments(moments)

dst = M{mu02: سنجش کجی,mu03: ,m00:مساحت مستطیل ,m10: ,m01: ,m11:سنجش کجی ,.....}آرایه شامل مومنت های مکانی و مرکزی و مرکزی نرمال

arr = تصویر ورودی که باید اشیاع درون آن از زمینه روشن تر باشند

تابع دوم از خروجی اولی استفاده میکندومومنت های کمتری خروجی میدهد

مفهوم مومنت ها به نسبت انهاست

cx = int (M["m10"]/M["m00"])#xمرکزشئ

cy = int (M["m01"]/M["m00"])#yمرکز شئ

In [2]:
import numpy as np
import cv2

font = cv2.FONT_HERSHEY_SIMPLEX
image = cv2.imread("R.jpg")
blurred = cv2.GaussianBlur(image,(7,7),0)
filtered = cv2.pyrMeanShiftFiltering(blurred,20,10)
gray = cv2.cvtColor(filtered ,cv2.COLOR_BGR2GRAY)          

_,thresh = cv2.threshold(gray,150,255,cv2.THRESH_BINARY)
thresh = cv2.morphologyEx(thresh,cv2.MORPH_OPEN,None,iterations=1)

_,labels = cv2.connectedComponents(thresh,connectivity=4,ltype=cv2.CV_32S)#شماره گذاری اجسام داخل تصویر 
#print(labels.shape)
#print(np.where(labels==5))

mask = np.zeros(thresh.shape,np.uint8)
center = []
for label in np.unique(labels):
    if label==0:
        continue
    labelMask = np.zeros(thresh.shape,np.uint8)
    labelMask[labels == label] = 255
   # labelMask = labels.copy()
    #labelMask = np.where(labelMask==label,255,0)
   # print(labelMask)
    numPixels = cv2.countNonZero(labelMask)
   # cv2.imshow("labelmask",labelMask)
    #cv2.waitKey(0)
    
    if numPixels>2000:
        M = cv2.moments(labelMask)
        cx = int(M["m10"]/M["m00"])
        cy = int(M["m01"]/M["m00"])
        center.append((cx,cy))
        mask = cv2.add(mask,labelMask)
for i,center in enumerate(center):
    cv2.circle(image,center,30,(32,100,255),2)
    cv2.putText(image,"{}".format(i+1),center,font,1,0,3)
    
cv2.imshow("Detect Coin",image)
cv2.waitKey(0)
cv2.destroyAllWindows()
        


# _,labels = cv2.connectedComponents(thresh,connectivity=8,ltype=cv2.CV_32S)


شماره گذاری اجسام سفید رنگ در ماسک

# np.unique(labels)

تعداد برچسب ها را برمیگرداند

In [2]:
import numpy as np
import cv2
from skimage import io

sz = 50
affin_flags = cv2.WARP_INVERSE_MAP | cv2.INTER_LINEAR
image = io.imread_collection("A*.jpg")

for img in image:
    img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    h,w = img.shape
    _,thresh = cv2.threshold(img,127,255,cv2.THRESH_BINARY_INV)
    m = cv2.moments(thresh)
    if abs(m['mu02'])<0.01:
        print("image has no skew")
    else:
        skew = m['mu11']/m['mu02']
        print(skew)
        M = np.float32([[1,skew,(-0.5*sz*skew)],[0,1,0]])
        img2 = cv2.warpAffine(thresh,M,(w,h),affin_flags)
        
        _,res = cv2.threshold(img2,127,255,cv2.THRESH_BINARY_INV)
        cv2.imshow("After Deskewed",res)
        cv2.waitKey(0)
cv2.destroyAllWindows()        
    

-0.31865355004419926
0.3541041593958058


مومنت همیشه ماسک سفید را از زمینه مشکی تشخیص میدهد و کلا باید جسم از زمینه روشن تر باشد اگر زمینه سفید بود وماسک مشکی

با متد آستانه گیری مورد نظر آنرا برعکس میکنیم

# کانتورها

داده مجموعه ای از نقاط را برمیگرداندFindContoursرا به Canny خروجی تابع 

{(m0,1,2),(m1,1,2),(m2,1,2),....}#خروجی تابع کانتور

هرپرانتز گویای یک جسم است وهرکدام بیانگراین است که ماتریس هرنقطه دارای یک ستون است که هرسطر  دو مقدار درخود دارد (عرض وارتفاع) 



# (image,contours,hierarchy) = cv2.findContours(image,mode,method)

image=تصویر ورودی 8 بیتی و تک کاناله -خروجی تابع کنی یا آستانه گیرکه خروجی آستانه گیر باید اجسام سفید باشند

mode=این پارامتر تعیین میکند کدام کانتورها انتخاب شوند

RETR_EXTERNALپیداکردن خارجی ترین کانتور

RETR_LISTپیدا کردن تمام کانتورها صرف نظراز رابطه بینشان

RETR_CCOMPپیداکردن تمام کانتورها ودسته بندی آنها از خارجی ترین به داخلی ترین

RETR_TREEمانند متد قبل ولی بهینه تر

method=نحوه تقریب کانتورها

CHAIN_APPROX_NONEتمام نقاط تشکیل دهنده هر کانتور را ذخیره میکند

CHAIN_APPROX_SIMPLEرئوس یا نقاط مهم هر کانتور راه نگه میدارد

contours=کانتورهای یافت شده بصورت مجموعه ای از نقاط

hierarchy=رابطه بین کانتورهای یافت شده را در خود نگه میدارد

# cv2.drawContours(image,contours,contourIdx,color,thickness,lineType,hierarchy,maxLevel)

image=تصویری که قرار است کانتورها روی آن رسم شود

contours=findcontour خروجی تابع 

contourIdx=اندیس کانتورهای که میخواهیم رسم شود(اگر1-بشد تمامش رسم میشود)

color=رنگ کانتورها 

thickness=ضخامت کانتورها(1-توپر میشود)

lineType=کیفیت رسم کانتورها

hierarchy=بدست می آیدfindcontourرابطه بین کانتورهاکه از تابع 

maxLevel=این پارامتر مشخص میکند چند دسته کانتور رسم شود(خارجی ترین-داخلی ها-داخلی داخلی هاو....)



In [3]:
import cv2
import numpy as np

image = cv2.imread("R.jpg")
clone = image.copy()
#image = cv2.resize(image,None,fx=0.5,fy=0.5,interpolation=cv2.INTER_AREA)
gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
blurred = cv2.GaussianBlur(gray,(7,7),0)
edged = cv2.Canny(blurred,30,150)

mode = cv2.RETR_EXTERNAL
#method = cv2.CHAIN_APPROX_SIMPLE
method = cv2.CHAIN_APPROX_NONE

cnts,_ = cv2.findContours(edged.copy(),mode,method)
#print(len(cnts))
cv2.drawContours(image,cnts,-1,(0,255,0),3)
#print((cnts[1]))
font = cv2.FONT_HERSHEY_SIMPLEX
for i,c in enumerate (cnts):
    M = cv2.moments(c)
#    cx = int(M['m10']/M['m00'])
#    cy = int(M['m01']/M['m00'])
#    cv2.putText(image,f"{i}",(cx,cy),font,1,200,1)
cv2.imshow("Coins",image)
cv2.waitKey(0)
cv2.destroyAllWindows()


مشکلی که مثال بالا دارد این است که بعضی از کانتورها مساحت 0 دارند و هنگام بدست آوردن مرکزشان به مشکل میخوریم

In [34]:
tim1 = time.time()*1000

In [39]:
tim2 = time.time()*1000

# تمرین

In [6]:
import cv2
import numpy as np
import imutils
from matplotlib import pyplot as plt
GAUSSIAN_SMOOTH_FILTER_SIZE = (1, 1)
ADAPTIVE_THRESH_BLOCK_SIZE = 19
ADAPTIVE_THRESH_WEIGHT = 9

img = cv2.imread("C:\\Users\\Admin\\OpenCv\\a\\p.png")

gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
img = cv2.resize(img,None,fx=0.8,fy=0.8,interpolation=cv2.INTER_AREA)

#blurred = cv2.GaussianBlur(gray,(3,3),0)
#cv2.imshow("Original",blurred)

#gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
#gradx = cv2.Sobel(gray,cv2.CV_32F,1,0,ksize=-1)
#grady = cv2.Sobel(gray,cv2.CV_32F,0,1,ksize=-1)#خروجی این خط و خط بالا اعداد شناورند
#sub = cv2.subtract(grady,gradx)#این دستور خروجی ها را ازهم کم میکند
#gradient = cv2.convertScaleAbs(sub)# این دستور خروجی بالا را به داده های 8 بیتی تبدیل میکند
#cv2.imshow("Gradients subtract",gradient)
    #cv2.waitKey(0)
    
#blurred = cv2.blur(gradient,(9,9))
#thresh = cv2.threshold(blurred,127,255,cv2.THRESH_BINARY)[1]
    
#kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(21,7))
#closed = cv2.morphologyEx(thresh,cv2.MORPH_CLOSE,kernel)
##closed = cv2.erode(closed,None,iterations=4)
#closed = cv2.dilate(closed,None,iterations=4)

def maximizeContrast(img):

    height, width = img.shape

    imgTopHat = np.zeros((height, width, 1), np.uint8)
    imgBlackHat = np.zeros((height, width, 1), np.uint8)

    structuringElement = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))

    imgTopHat = cv2.morphologyEx(img, cv2.MORPH_TOPHAT, structuringElement)
    imgBlackHat = cv2.morphologyEx(img, cv2.MORPH_BLACKHAT, structuringElement)

    imgGrayscalePlusTopHat = cv2.add(img, imgTopHat)
    imgGrayscalePlusTopHatMinusBlackHat = cv2.subtract(imgGrayscalePlusTopHat, imgBlackHat)

    return imgGrayscalePlusTopHatMinusBlackHat
#cv2.imshow("def",maximizeContrast(gray))
def preprocess(imgOriginal):
    imgGrayscale = extractValue(imgOriginal)

    imgMaxContrastGrayscale = maximizeContrast(imgGrayscale)

    height, width = imgGrayscale.shape

    imgBlurred = np.zeros((height, width, 1), np.uint8)

    imgBlurred = cv2.GaussianBlur(imgMaxContrastGrayscale, GAUSSIAN_SMOOTH_FILTER_SIZE, 0)

    imgThresh = cv2.adaptiveThreshold(imgBlurred, 255.0, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, ADAPTIVE_THRESH_BLOCK_SIZE, ADAPTIVE_THRESH_WEIGHT)

    return imgGrayscale, imgThresh
def extractValue(imgOriginal):
    height, width, numChannels = imgOriginal.shape

    imgHSV = np.zeros((height, width, 3), np.uint8)

    imgHSV = cv2.cvtColor(imgOriginal, cv2.COLOR_BGR2HSV)

    imgHue, imgSaturation, imgValue = cv2.split(imgHSV)

    return imgValue

#OP = [("MORPH_CLOSE",cv2.MORPH_CLOSE)]
#kernel = np.ones((3,3),np.int8)
#kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(3,3))
#morph = cv2.morphologyEx(img,OP[0][1],kernel,iterations=3)
#thresh,dst1 = cv2.threshold(img,70,200,cv2.THRESH_BINARY)
#morph = cv2.erode(img,kernel,iterations=5)

#kernel = np.ones((1,1),np.uint8)
dst1 = cv2.adaptiveThreshold(gray,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,29,-1)
#edged = cv2.Canny(dst1,10,100)
#dst1 = cv2.morphologyEx(dst1,cv2.MORPH_BLACKHAT,kernel,iterations=11)
#dst1 = cv2.erode(dst1,None,iterations=1)
cv2.imshow("thresh",dst1)
#edged = imutils.auto_canny(blurred,sigma=0.33)
(contours) = cv2.findContours(dst1,mode=cv2.RETR_TREE,method=cv2.CHAIN_APPROX_SIMPLE)[0]
#print(len(contours))
cnt = sorted(contours,key=cv2.contourArea,reverse=True)[0]
#cnt = max(contours,key=cv2.contourArea)#ماکزیمم مقادیر داده شده راباکلید مورد نظر پیدا میکند 
#print(cv2.contourArea(cnt))
#cv2.drawContours(dst1,contours,-1,(0,255,0),1)
#cv2.imshow("contours",dst1)
#for i,j in enumerate (contours):
#    p = cv2.arcLength(j,True)
#p = cv2.arcLength(cnt,True)
  #  approx = cv2.approxPolyDP(j,0.05*p,True)
  #  print(len(approx))
   # if len(approx)==4:
#rect = cv2.minAreaRect(cnt)
#box =np.int0 (cv2.boxPoints(rect))
    #    x,y,w,h = cv2.boundingRect(j)
   #     print(img.shape)
   #     print(w)
   #     print(h)
   # if 200 < w < 400 and 50 < h:
        #L = (w/float(h))
        #print(f'x{i}: {w}',f'y{i}: {h}')
if cv2.contourArea(cnt)>7000:
    x,y,w,h = cv2.boundingRect(cnt)
    cv2.rectangle(img,(x,y),(x+w,y+h),(255,255,0),2)
    #cv2.rectangle(img,box[0],box[2],(255,255,0),2)
#plt.subplot(111)
cv2.imshow('img',img)
#plt.title(f"{OP[0][0]}")
#plt.axis("off")
#plt.show() 
cv2.waitKey()
cv2.destroyAllWindows()

# ویژگی های کانتورها

-نقاط اکسترمم-پستی و بلندی های روی کانتور ها-مساحت-محیط

# contourArea مساحت کانتورها با 

 'm00' این تابع کانتور مورد نظر را ورودی میگیرد ومساحت محصور داخل آنرا بر حسب پیکسل خروجی میدهدمعادل  

#  arcLength محیط کانتوربا 

این تابع دارای 2 ورودی است اولین ورودی کانتور مورد نظر و دومین ورودی یک پرچم است که نشان میدهد کانتور باز است یا بسته

# تشخیص رنگ اجسام

In [1]:
import cv2 
import numpy as np
import imutils as im
from scipy.spatial import distance as dist

def dictionary():
    colors = {"blue":(255,0,0),"green":(0,255,0),"red":(0,0,255)}
    lab = np.zeros((len(colors),1,3),dtype='uint8')
    colorName = []
    
    for (i,(name,value)) in enumerate (colors.items()):
        lab[i] = value
        colorName.append(name)    
    lab = cv2.cvtColor(lab,cv2.COLOR_BGR2LAB)
    return colorName,lab

def color_label(image,contour):
    mask = np.zeros(image.shape[:2],dtype="uint8")
    cv2.drawContours(mask,[contour],-1,255,-1)
    mask = cv2.erode(mask,None,iterations=2)
    mean = cv2.mean(image,mask=mask)[:3]#خروجی 5مقدار دارد که 2تای آخری صفر است
   # print(mean)
    minDist = (np.inf,None) #(خالی و بینهایت)
    (colorName,lab) = dictionary()
    
    for (i,row) in enumerate (lab):
       # print(row)
        d = dist.euclidean(row[0],mean)
        #print(minDist[0])
        if d < minDist[0]:
            minDist = (d,i)
    return colorName[minDist[1]] 


image = cv2.imread("OIP.jpg")
#print(cv2.mean(image))
blurred = cv2.GaussianBlur(image,(5,5),0)
image_gray = cv2.cvtColor(blurred,cv2.COLOR_BGR2GRAY)
image_lab = cv2.cvtColor(blurred,cv2.COLOR_BGR2LAB)

_,thresh = cv2.threshold(image_gray,180,255,cv2.THRESH_BINARY_INV)
cnts = []
mode = cv2.RETR_EXTERNAL
method = cv2.CHAIN_APPROX_SIMPLE
contours,_ = cv2.findContours(thresh,mode,method)
#print(contours)
#print(len(contours))
for c in contours:
    if cv2.contourArea(c) >=100:
        cnts.append(c)

font = cv2.FONT_HERSHEY_SIMPLEX
for c in cnts:
    color_name = color_label(image_lab,c)
    M = cv2.moments(c)
    cx = int(M['m10']/M['m00'])
    cy = int(M['m01']/M['m00'])
    cv2.drawContours(image,[c],-1,(0,255,0),2)
    
    text ="{}".format(color_name)
    cv2.putText(image,text,(cx,cy),font,0.5,(255,255,255),2)
    
cv2.imshow("Color Detected",image)
cv2.waitKey(0)
cv2.destroyAllWindows()
    

(array([[[565, 456]],

       [[564, 457]],

       [[558, 457]],

       [[557, 458]],

       [[553, 458]],

       [[552, 459]],

       [[549, 459]],

       [[548, 460]],

       [[545, 460]],

       [[544, 461]],

       [[541, 461]],

       [[540, 462]],

       [[538, 462]],

       [[537, 463]],

       [[535, 463]],

       [[534, 464]],

       [[533, 464]],

       [[532, 465]],

       [[530, 465]],

       [[529, 466]],

       [[528, 466]],

       [[527, 467]],

       [[526, 467]],

       [[525, 468]],

       [[524, 468]],

       [[523, 469]],

       [[522, 469]],

       [[521, 470]],

       [[520, 470]],

       [[519, 471]],

       [[518, 471]],

       [[517, 472]],

       [[516, 472]],

       [[515, 473]],

       [[514, 473]],

       [[512, 475]],

       [[511, 475]],

       [[510, 476]],

       [[509, 476]],

       [[507, 478]],

       [[506, 478]],

       [[502, 482]],

       [[501, 482]],

       [[483, 500]],

       [[483, 501]],

       [[

# نکته:ورودی تابع فایند کانتور باید ماسک با زمینه مشکی باشد واجسام سفید باشند
    یاهم خروجی تابع کنی بعنوان ورودی داده شود
    
    نکته:در مثال بالا بادستور میانگین و ماسکی که از روی کانتور تولید شده ،میانگین پیکسل ها را روی تصویر جایی که ماسک حضور دارد بدست می آوریم
    
    با استفاده از مساحت کانتور،مساحتی که کوچک باشد را رد میکنیم
    
    سپس با مقایسه برداری خروجی میانگین و ماتریس رنگ های قرمز وآبی وسبز(درفضای لب)رنگ هر کانتور را تشخیص میدهیم
    
    

In [8]:
help(cv2.mean)

Help on built-in function mean:

mean(...)
    mean(src[, mask]) -> retval
    .   @brief Calculates an average (mean) of array elements.
    .   
    .   The function cv::mean calculates the mean value M of array elements,
    .   independently for each channel, and return it:
    .   \f[\begin{array}{l} N =  \sum _{I: \; \texttt{mask} (I) \ne 0} 1 \\ M_c =  \left ( \sum _{I: \; \texttt{mask} (I) \ne 0}{ \texttt{mtx} (I)_c} \right )/N \end{array}\f]
    .   When all the mask elements are 0's, the function returns Scalar::all(0)
    .   @param src input array that should have from 1 to 4 channels so that the result can be stored in
    .   Scalar_ .
    .   @param mask optional operation mask.
    .   @sa  countNonZero, meanStdDev, norm, minMaxLoc



# ناهمواری های کانتور

پیکسل هایی که به مرکز کانتور نزدیک هستند فرورفتگی و پیکسل هایی که از مرکز کانتور دور هستند را برآمدگی گویند

 ناهمواری های کانتور برابر است با مجموع فرورفتگی و برآمدگی ها

# cv2.isContourConvex(c)= True or False

این تابع با ورودی گرفتن کانتور اعلام میکند کانتور برآمدگی دارد یا خیر

# hull = cv2.convexHull (points,hull,clockWise,returnPoints)

این تابع مختصات یا اندیس نقاط برآمده برا خروجی میدهد

points = تنها یک کانتور را ورودی میگرد

hull =  گذاشته میشودNoneهمان خروجی تابع است که 

clockWise =باشد(False)یا پادساعتگرد (True) این پارامتر مشخص میکند برآمدگی ها بصورت ساعتگرد 

returnPoints =باشد مختصات نقاط وگرنه اندیس آنها را خروجی میدهدTrue اگر

hull =مختصات یا اندیس برآمدگی ها روی کانتور ورودی





# (start,end,def,dist) = cv2.convexityDefects(contour,convexHull)

این تابع اندیس فرورفتگی ها برمیگرداند

contour =  کانتوری که ورودی تابع قبلی داده شده

convexHull = اندیس نقاط برآمده

start =اندیس برآمدگی شروع

end =اندیس برآمدگی بعد از شروع

def =اندیس فرورفتگی بین دو برامدگی بالا

dist =فاصله فرورفتگی تا خطی که دو برآمدگی رل بهم وصل میکند



با بدست آوردن نقاط برآمدگی و فرورفتگی و دادن این نقاط به توابع رسم دایره  یا رسم چند ضلعی میتوان این نقاط را روی تصویر نشان داد

مثال:استفده از دوتابع بالا

In [2]:
import cv2
import numpy as np

img = cv2.imread("hand.jpg")
img1 = img.copy()
img2 = img.copy()

gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

#edged = cv2.Canny(gray,100,150)
dst1 = cv2.adaptiveThreshold(gray,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,25,-1)

mode = cv2.RETR_EXTERNAL
method = cv2.CHAIN_APPROX_SIMPLE

#contours,_ = cv2.findContours(edged.copy(),mode,method)
contours,_ = cv2.findContours(dst1.copy(),mode,method)
#print(contours)
cnt = max(contours,key=cv2.contourArea)#ماکزیمم مقادیر داده شده راباکلید مورد نظر پیدا میکند 
#print(cnt)
#print(cnt[:,:,0])
#hull = cv2.convexHull(cnt)
hull = cv2.convexHull(cnt,None,returnPoints=False)
#print(hull)
defects = cv2.convexityDefects(cnt,hull)

#print(defects)
#print(defects.shape)

#for c in hull:
    #print(c)
  #  cv2.circle(img1,tuple(c[0]),10,(255,0,0),-1)#تنسوری است که هر ماتریس آن یک سطر ودو ستون داردhullخروجی تابع 
for i in range(defects.shape[0]):
    s,e,f,d = defects[i,0]
    start = tuple(cnt[s][0])
   # print(start)
    end = tuple(cnt[e][0])
    far = tuple(cnt[f][0])
    if (d>=10000 and d<100000):
        cv2.circle(img2,start,10,(0,255,0),-1)
        cv2.circle(img2,end,10,0,5)
        cv2.circle(img2,far,10,0,2)
    cv2.line(img2,start,end,0,2)   
#cv2.polylines(img1,[hull],True,0,2)
#cv2.imshow("Convex",img1)

cv2.imshow("Convex",img2)
cv2.waitKey(0)
cv2.destroyAllWindows()

خروجی فایند کانتورتاپلی است که به ازای هر کانتور پیدا شده یک تنسور میدهد که داخلش نقاط کانتور در قالب هر نقطه یک ماتریس نمایش داده میشوند

را نمایش میدهدstart , end, def ,dist یک تنسور است که هر ماتریس آن نقاط convexityDefectsخروجی

 نیز تنسور استapproxخروجی 
 
# از روی اندیس های یافت شده میتوان استفاده کرد و نقاط آنها را از تنسوری که فایند کانتور خروجی داده است بدست آورد   

# نقاط اکسترمم

منظور از نقاط اکسترمم،بالاترین و پایین ترین و چپ ترین و راست ترین نقاط کانتور است

cn[:,:,0].argmax()#راست ترین نقطه

cn[:,:,0].argmin()#چپ ترین نقطه

cn[:,:,1].argmax()#پایین ترین نقطه

cn[:,:,1].argmin()#بالا ترین نقطه

حال که اندیس ماتریس ها را پیدا کردیم

از تنسور کانتور مورد نظر نقاط هر اندیس را پیدا میکنیم

left_most = tuple(cnt[cnt[:,:,0].argmin()][0])

right_most = tuple(cnt[cnt[:,:,0].argmax()][0])

top_most = tuple(cnt[cnt[:,:,1].argmin()][0])

bottom_most = tuple(cnt[cnt[:,:,1].argmax()][0])

# تقریب کانتور با اشکال هندسی

در برخی موارد ممکن است قسمت هایی از کانتور از بین رفته باشند و کانتور شکل اصلی خود را ازدست بدهد در این حالت میتوان شکل هندسی آنرا حدس زد

# چند ضلعی غیر منتظم

pts = cv2.approxPolyDP(curve,epsilon,closed)

curve =  نقاط کانتور 

epsilon = میزان دقت الگوریتم،شباهت بین منحنی اصلی و تقریبی را تعیین میکندبراساس محیط کانتور ورودی مشخص میشود

closed = باز یا بسته بودن منحنی تقریبی را مشخص میکند

pts = نقاط تشکیل دهنده منحنی تقریبی





مثال:تشخیص اشکال هندسی

In [1]:
import cv2
import numpy as np


image = cv2.imread("shapes.jpg")
gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)

_,dst1 = cv2.threshold(gray,230,255,cv2.THRESH_BINARY_INV)#از این روش استفاده شدmomentsبرای عملکرد

#kernel=np.ones((3,3),dtype="uint8")
#erode = cv2.dilate(dst1,kernel,iterations=3)

#cv2.imshow("",dst1)
#cv2.waitKey(0)


mode = cv2.RETR_EXTERNAL
method = cv2.CHAIN_APPROX_SIMPLE

cnt,_ = cv2.findContours(dst1,mode,method)
font = cv2.FONT_HERSHEY_SIMPLEX
shape = None
#print(cnt)
for c in cnt:
    p = cv2.arcLength(c,True)
    approx = cv2.approxPolyDP(c,0.04*p,True)
   # print(approx)
    if len(approx) == 3 :
        shape = "Triangle"
        
    elif len(approx) == 4 :
        (x,y,w,h) = cv2.boundingRect(approx)
        L = (w/float(h))
        if (L>=0.95 and L<=1.05):
            shape = "Square"   
        else:
            shape = "Rectangle"
            
    elif len(approx) == 5:
        shape = "Pentagon"
        
    else:
        shape = "Circle"
        
    M = cv2.moments(c)
    cx = int(M["m10"]/M["m00"])
    cy = int(M["m01"]/M["m00"])
    cv2.putText(image,shape,(cx,cy),font,0.8,0,3)
    
cv2.imshow("Shape Detection",image)
cv2.waitKey(0)
cv2.destroyAllWindows()

# چهار ضلعی ثابت و دورانی

# (x,y,w,h) = cv2.boundingRect(inputArray) #ثابت

inputArray = مجموعه نقاط دوبعدی یا همان کانتور ورودی

(x,y)= مختصات گوشه سمت چپ وبالا چهارضلعی پیدا شده

(w,h)= عرض و ارتفاع چهار ضلعی پیدا شده

این تابع کانتور را باکوچکترین چهار ضلعی که بتواند کانتور را درخود جای دهد تخمین میزند

# (x,y)(w,h),angle = cv2.minAreaRect(inputArray)# دورانی(دقیق)

inputArray = مجموعه نقاط دوبعدی یا همان کانتور ورودی

(x,y)= مختصات گوشه سمت چپ وبالا چهارضلعی پیدا شده

(w,h)= عرض و ارتفاع چهار ضلعی پیدا شده

angle = مقدار زاویه دوران مستطیل

# داده تا 4 نقاط گوشه های مستطیل را بدهدboxPoints به تابع minAreaRect نکته: خروجی تابع 

[[x0,y0],[x1,y1],[x4,y4],[x3,y3]] = np.int0(cv2.boxPoints(rect))  #          rectخروجی تابع بالا

مثال:تشخیص جهت اشاره انگشت بصورت بلادرنگ

In [5]:
import cv2
import numpy as np
import imutils
from scipy.spatial import distance as dist
import math

def find_pointer(cnt,center):#تابع مختصات نوک انگشت را پیدا میکند و خروجی میدهد
    left = tuple(cnt[cnt[:,:,0].argmin()][0])
    right = tuple(cnt[cnt[:,:,0].argmax()][0])
    top = tuple(cnt[cnt[:,:,1].argmin()][0])
    bottom = tuple(cnt[cnt[:,:,1].argmax()][0])
    extreme = (left,right,top,bottom)
    
    result = []
    for ext in (extreme):
        d = dist.euclidean(ext,center)#فاصله اقلیدسی دومقدار را میدهد
        result.append(d)
    pointer = [extreme for _,extreme in sorted(zip(result,extreme),reverse = True) ][0]
    return pointer


def navigation(c,p):#تابع تعیین جهت انگشت
    m = None
    theta = None
    direction = None
    if abs(p[0]-c[0])==0:#ها برابر با صفر باشدیعنی مرکز با اشاره گر روی محور عمودی در یک راستا هستندxاگر قدر مطلق تفاضل 
        if (p[1]<center[1]):# اشاره گر کمتر از مرکز باشدیعنی اشاره به سمت شمال استyاگر 
            direction = "Sorth"
            theta = 90
        elif (p[1]>c[1]):#اشاره گر بیشتر از مرکز باشدیعنی اشاره به سمت شمال استyاگر 
            direction = "North"
            theta = 270 #زاویه بصورت پادساعتگرد استفاده میشود
            
    else:
        m = abs((p[1]-c[1])/float(p[0]-c[0]))#زوایا را فقط بین0تا90در نظر میگیردatanدلیل استفاده از قدر مطلق این است که تابع
        theta = (math.atan(m)*180)/math.pi#تابع روبرو برای محاسبه زاویه شیب خط استفاده میشودکه از رادیان به درجه بازمیگردانیم
        
        if ((p[1]<c[1]) and (p[0]<c[0])):
            theta = theta
        elif ((p[1]<c[1]) and (p[0]>c[0])):
            theta = 90+(90-theta)
        elif ((p[1]>c[1]) and (p[0]>c[0])):
            theta = 180+theta
        elif ((p[1]>c[1]) and (p[0]<c[0])):
            theta = 270+(90-theta)
    if (theta<15 or theta>=345):#پیداکردن جهت از روی زاویه بدست آمده
        direction = "West"
    elif (theta>=15 and theta<75):
        direction = "West North"
    elif (theta<=75 and theta<95):
        direction = "North"
    elif (theta>=95 and theta<165):
        direction = "East north"
    elif (theta>=165 and theta<185):
        direction = "East"
    elif (theta>=185 and theta<255):
        direction = "East south"
    elif (theta>=255 and theta<275):
        direction = "South"
    elif (theta>=275 and theta<345):
        direction = "West South" 
    return direction

cam = cv2.VideoCapture(0)
yellow_lower = (0,100,100)#حداقل رنگ زرد
yellow_upper = (120,255,255)#حداکثر رنگ زرد
kernel = np.ones((10,10),np.uint8)
font = cv2.FONT_HERSHEY_TRIPLEX
while (True):
    _,frame = cam.read()
    frame = cv2.flip(frame,1)#تصویر را قرینه میکند
    clone = frame.copy()
    
    blurred = cv2.GaussianBlur(frame,(5,5),0)
    frame_hsv = cv2.cvtColor(blurred,cv2.COLOR_BGR2HSV)#میبریم hsvبرای تشخیص درست رنگ ها به فضای 
    mask = cv2.inRange(frame_hsv,yellow_lower,yellow_upper)#ماسکی درست میکنیم با پیکسل های زرد
    mask = cv2.morphologyEx(mask,cv2.MORPH_OPEN,kernel,iterations=1)#با تبذیل ریختی ماسک را دقیق تر کرده
    
    mode = cv2.RETR_EXTERNAL
    method = cv2.CHAIN_APPROX_SIMPLE
    (cnts,_) = cv2.findContours(mask.copy(),mode,method)#کانتور ماسک جسم زرد رنگ را بدست آورده
    if len(cnts)>0:
        cnt = max(cnts,key=cv2.contourArea)#بزرگترین کانتور را انتخاب کرده
        x,y,w,h = cv2.boundingRect(cnt)#چهارضلعی محصور دورش را حدس زده
        xS = x-20
        yS = y-20
        xE = x+w+20
        yE = y+h+20
        cv2.rectangle(frame,(xS,yS),(xE,yE),(0,255,0),3)
        M = cv2.moments(cnt)
        cx = int(M["m10"]/M["m00"])
        cy = int(M["m01"]/M["m00"])
        center = (cx,cy)
        cv2.circle(frame,center,10,(0,0,255),-1)
        
        pointer=find_pointer(cnt,center)
        cv2.circle(frame,pointer,10,(0,0,255),-1)
        direction = navigation(center,pointer)
        cv2.putText(frame,"{}".format(direction),(20,50),font,2,(0,0,255),2)
        cv2.imshow("Original frame",clone)
        cv2.imshow("Masking",mask)
        cv2.imshow("Navigation",frame)
        if cv2.waitKey(1)==27:
            break
cam.release()
cv2.destroyAllWindows()
        

# m = y2-y1/x2-x1#شیب خط
# radian=math.atan(m)#پیداکردن زاویه از شیب خط بر حسب رادیان
# theta = radian*180/math.pi #تبدیل رادیان به درجه

 زوایا را بین 0و90 درجه بدست می آورد که ما میتوانیم بامحاسبه اینکه نقطه ما درکجای دایره قرار گرفته اعمال زیر را انجام دهیمatan تابع 
 
 نیمکره اول= tetha is ok  
 
 نیمکره دوم= tetha ==  90+(90-theta)
 
 نیمکره سوم= tetha ==  180+(theta)
 
 نیمکره چهارم= tetha == 270+(90-theta)
 
 

مثال:اسکن مدارک با تشخیص خودکار سند

In [ ]:
import cv2
import numpy as np
import imutils as im

def find_paper(image):#تابع برای پیدا کردن کاغذ
    flag = False
    contour = 0
    points = 0
    C_mode = cv2.RETR_EXTERNAL
    C_method = cv2.CHAIN_APPROX_SIMPLE
    
    blurred = cv2.GaussianBlur(image,(3,3),0)#حذف نویز
    edged = cv2.Canny(blurred,50,100)#لبه یابی
    dilate = cv2.dilate(edged,None,iterations=1)#گسترش بخش سفید
    cnts = cv2.findContours(dilate,C_mode,C_method)[0]#کانتوریابی
    
    if len(cnts)!= 0:
        cnt = sorted(cnts,key=cv2.contourArea,reverse=True)[0]#خروجی بزرگترین کانتور
        if (cv2.contourArea(cnt)>=200):
            peri = cv2.arcLength(cnt,True)
            approx = cv2.approxPolyDP(cnt,0.02*peri,True)#پیداکردن رئوس کانتور
            if len(approx) == 4:
                flag = True
                contour = cnt
                points = approx
               # print(points)
    return (flag,contour,points)



def order_points(points):#مشخص میکند هر نقطه مربوط به کدام گوشه کاغذ است
    y_sorted = points[np.argsort(points[:,0,1])]#اندیس نقاط را بعد از مرتب کردن برمیگرداند وسطر اول اندیس ها را اسلایس میکندargsortبا
    up_most = y_sorted[:2,:]
    down_most = y_sorted[2:,:]
    
    tl = up_most[np.argmin(up_most[:,:,1]),0]
    tr = up_most[np.argmax(up_most[:,:,1]),0]
    bl = down_most[np.argmin(down_most[:,:,1],0)]
    br = down_most[np.argmax(down_most[:,:,1],0)]
    return tl,tr,br,bl


def points_transform(image,points):#این تابع تصویر را prespectiveمیکند
    (tl,tr,br,bl) = order_points(points)
    pts = np.array([tl,tr,br,bl],dtype="float32")
    w = np.aqrt(((tl[1]-tr[1])**2)+((tl[0]-tr[0])**2))
    h = np.aqrt(((tl[1]-bl[1])**2)+((tl[0]-bl[0])**2))
    dts = np.array([[0,0],[w-1,0],[w-1,h-1],[0,h-1]],dtype="float32")
    
    M = cv2.getPerspectiveTransform(pts,dts)
    warped = cv2.warpPerspective(image,M,(int(w),int(h)))
    return warped



def scan(paper):
    global printT,Smethod,blockSize,C
    global dim,n_erode,n_dilate
    global coef,angle,border
    
    if printT==0:
        output = paper
    elif printT == 1:
        output = cv2.cvtColor(paper,cv2.COLOR_BGR2GRAY)
    elif printT==2:
        gray = cv2.cvtColor(paper,cv2.COLOR_BGR2GRAY)
        mode = cv2.THRESH_BINARY
        output = cv2.adaptiveThreshold(gray,255,Smethod,mode,blockSize,C)
        
    if dim==0:
        kernel = None
    
    else:
        kernel = cv2.getStructuringElementt(cv2.MORPH_RECT,(dim,dim))
    if(n_erode > 0):
        output = cv2.erode(output,kernel,iterations=n_erode)
    if (n_dilate>0):
        output = cv2.dilate(output,kernel,iterations=n_dilate)
        
    resize = cv2.resize(output,None,fx = coef,fy = coef,interpolation=cv2.INTER_AREA)
    rotated = im.rotate_bound(resize,angle*90)
    
    if border==1:
        dime = rotated.shape
        h = dime[0]
        w = dime[1]
        if len(dime) == 2:
            size = (h+10,w+10)
            border = np.zeros(size,dtype="uint8")
        elif len(dime) == 3:
            size = (h+10,w+10,3)
            border = np.zeros(size,dtype="uint8")
        border[0:h,0:w] = rotated
        final = im.translate(border,5,5)
    else:
        final = rotated
    return final


def nothing(x):
    pass

cam = cv2.VideoCapture(0)
paper = None

while(True):
    k = cv2.waitKey(1)
    _,frame = cam.read()
    clone = frame.copy()
    (flag,cnt,points) = find_paper(clone)
    if (flag==True):
        cv2.drawContours(frame,[cnt],-1,(0,0,255),2)
        if k== ord("s"):
            paper = points_transform(clone,points.reshape(4,2))
            cv2.imshow("Paper",paper)
            
            cv2.namedWindow("Setting",cv2.WINDOW_NORMAL)
            cv2.createTrackbar("Print Type","Setting",0,2,nothing)
            cv2.createTrackbar("Scan method","Setting",0,1,nothing)
            cv2.createTrackbar("block Size","Setting",7,200,nothing)
            cv2.createTrackbar("C param","Setting",5,50,nothing)
            cv2.createTrackbar("Kennel","Setting",0,10,nothing)
            cv2.createTrackbar("Erode iter","Setting",0,10,nothing)
            cv2.createTrackbar("Dilate iter","Setting",0,10,nothing)
            cv2.createTrackbar("Rotation","Setting",0,4,nothing)
            cv2.createTrackbar("Resize","Setting",0,5,nothing)
            cv2.createTrackbar("Border","Setting",0,1,nothing)
        if (paper is not None):
            printT = cv2.getTrackbarPos("Print Type","Setting")
            Smethod = cv2.getTrackbarPos("Scan method","Setting")
            blockSize = cv2.getTrackbarPos("block Size","Setting")
            
            if blockSize %2 ==0:
                blockSize = blockSize + 1
            C = cv2.getTrackbarPos("C param","Setting")
            dim = cv2.getTrackbarPos("Kennel","Setting")
            n_erode = cv2.getTrackbarPos("Erode iter","Setting")
            n_dilate = cv2.getTrackbarPos("Dilate iter","Setting")
            coef = cv2.getTrackbarPos("Resize","Setting")
            
            if coef == 0:
                coef = 1 
            angle = cv2.getTrackbarPos("Rotation","Setting")
            border = cv2.getTrackbarPos("Border","Setting")
            
            result = scan(paper)
            cv2.imshow("Paper",result)
            
            if k == ord("w"):
                cv2.imwrite("paper.png",result)
            elif k == ord("r"):
                paper = None
                cv2.destroyWindow("Paper")
                cv2.destroyWindow("Setting")
                
        if k == 27:
            break
        cv2.imshow("Original frame",frame)
cam.release()
cv2.destroyAllWindows()
                
            

# np.argsort(arr) #آرایه را مرتب میکند سپس اندیس های قبلی مقادیر را بعد از مرتب کردن  خروجی میدهد

مثال :بارکدخوان بلادرنگ

In [1]:
import numpy as np
import cv2
from pyzbar.pyzbar import decode

def barcod_detection(image):
    gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    gradx = cv2.Sobel(gray,cv2.CV_32F,1,0,ksize=-1)
    grady = cv2.Sobel(gray,cv2.CV_32F,0,1,ksize=-1)#خروجی این خط و خط بالا اعداد شناورند
    sub = cv2.subtract(grady,gradx)#این دستور خروجی ها را ازهم کم میکند
    gradient = cv2.convertScaleAbs(sub)# این دستور خروجی بالا را به داده های 8 بیتی تبدیل میکند
    cv2.imshow("Gradients subtract",gradient)
    #cv2.waitKey(0)
    
    blurred = cv2.blur(gradient,(9,9))
    thresh = cv2.threshold(blurred,127,255,cv2.THRESH_BINARY)[1]
    
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(21,7))
    closed = cv2.morphologyEx(thresh,cv2.MORPH_CLOSE,kernel)
    closed = cv2.erode(closed,None,iterations=4)
    closed = cv2.dilate(closed,None,iterations=4)
    cv2.imshow("Mask",closed)
   # cv2.waitKey(0)
    
    mode = cv2.RETR_EXTERNAL
    method = cv2.CHAIN_APPROX_SIMPLE
    cnts = cv2.findContours(closed.copy(),mode,method)[0]
    if len(cnts) == 0:
        return None
    cnt = sorted(cnts,key=cv2.contourArea,reverse=True)[0]
    rect = cv2.minAreaRect(cnt)
   # print(rect,'\n')
    box =np.int0 (cv2.boxPoints(rect))#
    print(box)
    return box
    
def order_points(points):
    border = 20
    y_sorted = points[np.argsort(points[:,1]),:]
    up_most = y_sorted[:2,:]
    down_most = y_sorted[2:,:]
    
    tl = up_most[np.argmin(up_most[:,0]),:]
    tl = (tl[0]-border,tl[1]-border)
    tr = up_most[np.argmax(up_most[:,0]),:]
    tr = (tr[0]+border,tr[1]-border)
    bl = down_most[np.argmin(down_most[:,0]),:]
    bl = (bl[0]-border,bl[1]+border)
    br = down_most[np.argmax(down_most[:,0]),:]
    br = (br[0]+border,br[1]+border)
   # print(tl,tr,br,bl)
    return[tl,tr,br,bl]

def points_transform(image,points):
    (tl,tr,br,bl) = order_points(points)
    pts = np.array([tl,tr,br,bl],dtype="float32")
    
    w = np.sqrt(((tl[1]-tr[1])**2)+((tl[0]-tr[0])**2))
    h = np.sqrt(((tl[1]-bl[1])**2)+((tl[0]-bl[0])**2))
    dts = np.array([[0,0],[w-1,0],[w-1,h-1],[0,h-1]],dtype="float32")
    
    M = cv2.getPerspectiveTransform(pts,dts)
    warped = cv2.warpPerspective(image,M,(int(w),int(h)))
    return warped


font = cv2.FONT_HERSHEY_SIMPLEX
cam = cv2.VideoCapture(0)
while(True):
    _,frame = cam.read()
    box = barcod_detection(frame)
    if box is not None:
        barcode = points_transform(frame,box.reshape(4,2))
        cv2.drawContours(frame,[box],-1,(0,255,0),2)
        
        data = decode(barcode)
        cv2.putText(frame,"Barcode: {}".format(data),(10,30),font,0.6,(0,0,255),2)
        cv2.imshow("Frame",frame)
        if cv2.waitKey(1)== 27:
            break
cam.release()
cv2.destroyAllWindows()

C:\Users\Admin\AppData\Local\Temp\ipykernel_1648\521514718.py:32: DeprecationWarning: `np.int0` is a deprecated alias for `np.intp`.  (Deprecated NumPy 1.24)
  box =np.int0 (cv2.boxPoints(rect))#


[[198 207]
 [200 118]
 [444 124]
 [442 212]]
[[200 122]
 [444 122]
 [444 212]
 [200 212]]
[[200 123]
 [444 123]
 [444 213]
 [200 213]]
[[201 126]
 [444 126]
 [444 216]
 [201 216]]
[[201 218]
 [201 128]
 [444 128]
 [444 218]]
[[199 117]
 [201  -2]
 [442   1]
 [440 121]]
[[198 117]
 [200  -2]
 [442   0]
 [440 121]]
[[199   0]
 [440   0]
 [440 121]
 [199 121]]
[[200 129]
 [444 129]
 [444 218]
 [200 218]]
[[199   0]
 [440   0]
 [440 121]
 [199 121]]
[[199   0]
 [440   0]
 [440 121]
 [199 121]]
[[197 115]
 [201  -5]
 [443   2]
 [439 123]]
[[201 122]
 [201   0]
 [441   0]
 [441 122]]
[[201 117]
 [204  -4]
 [443   1]
 [439 124]]
[[201   1]
 [442  -1]
 [445 219]
 [204 222]]
[[203   0]
 [442   0]
 [442 124]
 [203 124]]
[[203   0]
 [442   0]
 [442 123]
 [203 123]]
[[204   0]
 [446   0]
 [446 218]
 [204 218]]
[[205   0]
 [442   0]
 [442 121]
 [205 121]]
[[204   0]
 [443   0]
 [443 121]
 [204 121]]
[[203 120]
 [205  -2]
 [445   0]
 [443 123]]
[[206 121]
 [208  -3]
 [447   1]
 [444 125]]
[[208 136]

[[207 195]
 [208 123]
 [421 129]
 [419 200]]
[[207 196]
 [208 124]
 [420 128]
 [419 200]]
[[208 196]
 [209 124]
 [420 127]
 [419 199]]
[[207 195]
 [208 123]
 [421 127]
 [419 199]]
[[207 195]
 [208 123]
 [421 127]
 [419 199]]
[[207 194]
 [208 122]
 [420 127]
 [418 199]]
[[207 195]
 [208 122]
 [419 125]
 [418 198]]
[[206 194]
 [208 121]
 [422 127]
 [420 199]]
[[205 192]
 [207 120]
 [421 125]
 [419 198]]
[[206 193]
 [207 120]
 [421 125]
 [420 198]]
[[206 193]
 [207 119]
 [421 124]
 [420 198]]
[[205 191]
 [208 117]
 [422 125]
 [420 198]]
[[206 192]
 [207 118]
 [422 123]
 [420 197]]
[[205 190]
 [208 117]
 [423 124]
 [421 197]]
[[205 190]
 [208 117]
 [423 123]
 [420 197]]
[[206 188]
 [209 115]
 [424 124]
 [422 197]]
[[208 191]
 [210 118]
 [424 123]
 [422 197]]
[[208 191]
 [210 118]
 [424 123]
 [422 197]]
[[208 191]
 [211 117]
 [426 125]
 [423 198]]
[[210 192]
 [213 119]
 [426 125]
 [424 198]]
[[212 192]
 [213 119]
 [426 125]
 [424 197]]
[[211 190]
 [214 117]
 [426 124]
 [424 197]]
[[212 188]

[[218 192]
 [221 127]
 [410 135]
 [408 199]]
[[218 192]
 [221 127]
 [410 133]
 [408 199]]
[[112   0]
 [518   0]
 [518  70]
 [112  70]]
[[217 191]
 [221 107]
 [412 116]
 [407 200]]
[[217 191]
 [221 107]
 [411 115]
 [408 199]]
[[217 191]
 [219 126]
 [410 132]
 [408 198]]
[[216 191]
 [219 125]
 [410 132]
 [408 198]]
[[216 190]
 [220 106]
 [411 114]
 [408 198]]
[[216 190]
 [218 124]
 [410 131]
 [408 197]]
[[216 188]
 [219 122]
 [411 130]
 [408 196]]
[[218 110]
 [410 110]
 [410 194]
 [218 194]]
[[216 187]
 [219 121]
 [410 128]
 [408 194]]
[[216 185]
 [219 120]
 [410 129]
 [407 194]]
[[216 185]
 [219 120]
 [411 128]
 [409 193]]
[[216 186]
 [219 120]
 [411 126]
 [409 193]]
[[216 185]
 [219 118]
 [413 128]
 [409 194]]
[[217 185]
 [219 118]
 [412 126]
 [410 192]]
[[216 184]
 [221 100]
 [415 110]
 [410 194]]
[[217 186]
 [220 120]
 [414 126]
 [411 193]]
[[217 186]
 [220 119]
 [414 128]
 [412 195]]
[[218 186]
 [220 120]
 [414 128]
 [412 193]]
[[217 187]
 [220 120]
 [414 128]
 [412 194]]
[[218 186]

[[208 206]
 [210 139]
 [402 145]
 [400 212]]
[[208 204]
 [212 119]
 [408 130]
 [403 215]]
[[207 204]
 [213 118]
 [404 130]
 [399 216]]
[[208 206]
 [210 139]
 [402 145]
 [400 213]]
[[157   0]
 [512   0]
 [512  83]
 [157  83]]
[[125   0]
 [508   0]
 [508  84]
 [125  84]]
[[102   0]
 [509   0]
 [509  65]
 [102  65]]
[[207 206]
 [211 121]
 [402 129]
 [398 215]]
[[100  66]
 [103 -14]
 [510   5]
 [507  85]]
[[207 205]
 [211 120]
 [403 128]
 [400 214]]
[[100  52]
 [102 -10]
 [512   2]
 [510  66]]
[[208 208]
 [209 141]
 [403 144]
 [402 211]]
[[208 205]
 [211 133]
 [402 141]
 [400 213]]
[[102  53]
 [104  -9]
 [511   2]
 [509  65]]
[[101  53]
 [103  -9]
 [510   2]
 [508  65]]
[[209 207]
 [210 140]
 [403 145]
 [401 212]]
[[208 206]
 [211 121]
 [404 127]
 [401 213]]
[[208 205]
 [211 120]
 [404 128]
 [401 213]]
[[210  62]
 [216 -15]
 [515   6]
 [509  84]]
[[101   0]
 [512   0]
 [512  54]
 [101  54]]
[[102   0]
 [510   0]
 [510  62]
 [102  62]]
[[206 203]
 [209 136]
 [402 143]
 [400 210]]
[[206 203]

...

[[221 138]
 [392 138]
 [392 229]
 [221 229]]
[[ 30 226]
 [362  94]
 [431 268]
 [ 99 400]]
[[221  75]
 [224  -7]
 [485   1]
 [482  85]]
[[219 139]
 [391 134]
 [394 213]
 [221 218]]
[[302   0]
 [485   0]
 [485  84]
 [302  84]]
[[233   0]
 [487   0]
 [487  84]
 [233  84]]
[[253  75]
 [257  -7]
 [489   2]
 [486  85]]
[[218 143]
 [388 125]
 [397 213]
 [227 231]]
[[234  77]
 [238  -7]
 [491   3]
 [487  88]]
[[ 28 266]
 [ 67  90]
 [414 166]
 [375 342]]
[[235  78]
 [239  -5]
 [488   5]
 [485  89]]
[[177  75]
 [181  -8]
 [492   6]
 [488  91]]
[[222 145]
 [392 129]
 [400 215]
 [230 231]]
[[214   0]
 [490   0]
 [490 104]
 [214 104]]
[[214   0]
 [489   0]
 [489 104]
 [214 104]]
[[ 32 181]
 [390 123]
 [419 302]
 [ 62 361]]
[[260   0]
 [491   0]
 [491 106]
 [260 106]]
[[136   0]
 [491   0]
 [491 106]
 [136 106]]
[[ 39 241]
 [ 90  68]
 [409 163]
 [358 336]]
[[133  89]
 [140 -19]
 [495   2]
 [488 112]]
[[138 109]
 [138   0]
 [491   0]
 [491 109]]
[[137   0]
 [492   0]
 [492 111]
 [137 111]]
[[138   0]

[[266   0]
 [498   0]
 [498 111]
 [266 111]]
[[266   0]
 [499   0]
 [499 109]
 [266 109]]
[[266   0]
 [499   0]
 [499 109]
 [266 109]]
[[275   0]
 [498   0]
 [498 108]
 [275 108]]
[[261 110]
 [270 -15]
 [497   0]
 [488 125]]
[[269   0]
 [497   0]
 [497 108]
 [269 108]]
[[281   0]
 [498   0]
 [498 124]
 [281 124]]
[[234 235]
 [236 160]
 [409 164]
 [407 240]]
[[140   0]
 [501   0]
 [501 122]
 [140 122]]
[[234 233]
 [236 173]
 [409 178]
 [407 238]]
[[203   0]
 [501   0]
 [501 103]
 [203 103]]
[[199  74]
 [217 -35]
 [514  11]
 [497 120]]
[[203  88]
 [207  -9]
 [504   2]
 [500 100]]
[[139   0]
 [502   0]
 [502 112]
 [139 112]]
[[290   0]
 [501   0]
 [501  94]
 [290  94]]
[[290   0]
 [501   0]
 [501  94]
 [290  94]]
[[211   0]
 [499   0]
 [499  94]
 [211  94]]
[[308  28]
 [493 -18]
 [509  45]
 [325  92]]
[[131   0]
 [499   0]
 [499 112]
 [131 112]]
[[354   0]
 [497   0]
 [497  65]
 [354  65]]
[[361   0]
 [495   0]
 [495  97]
 [361  97]]
[[372   0]
 [495   0]
 [495 114]
 [372 114]]
[[307   0]

[[129   0]
 [504   0]
 [504 113]
 [129 113]]
[[129   0]
 [504   0]
 [504 111]
 [129 111]]
[[152 242]
 [157 141]
 [485 159]
 [480 260]]
[[ 33 240]
 [ 36 158]
 [574 174]
 [571 256]]
[[ 34 234]
 [ 37 140]
 [481 155]
 [478 249]]
[[ 35 231]
 [ 37 159]
 [490 173]
 [488 245]]
[[172 227]
 [177 142]
 [610 170]
 [605 254]]
[[148 222]
 [153 142]
 [609 172]
 [604 252]]
[[ 35 220]
 [ 38 141]
 [417 154]
 [414 234]]
[[ -1 174]
 [436 163]
 [438 225]
 [  0 237]]
[[312   0]
 [502   0]
 [502  95]
 [312  95]]
[[262   0]
 [501   0]
 [501  95]
 [262  95]]
[[185 216]
 [189 146]
 [441 162]
 [437 232]]
[[313   0]
 [503   0]
 [503 113]
 [313 113]]
[[129   0]
 [503   0]
 [503 112]
 [129 112]]
[[130   0]
 [504   0]
 [504  99]
 [130  99]]
[[ -3 164]
 [420 145]
 [423 226]
 [  0 244]]
[[ -1 174]
 [433 165]
 [434 227]
 [  0 236]]
[[133   0]
 [504   0]
 [504 114]
 [133 114]]
[[133   0]
 [504   0]
 [504 115]
 [133 115]]
[[313   0]
 [502   0]
 [502  99]
 [313  99]]
[[266   0]
 [502   0]
 [502 100]
 [266 100]]
[[300   0]

[[293  83]
 [343  21]
 [506 153]
 [455 215]]
[[290  87]
 [336  24]
 [502 147]
 [456 209]]
[[154 216]
 [233 161]
 [321 289]
 [241 343]]
[[166 188]
 [236 154]
 [310 309]
 [239 343]]
[[150 222]
 [234 158]
 [328 281]
 [243 345]]
[[162 220]
 [236 171]
 [317 294]
 [243 342]]
[[405 379]
 [461 351]
 [511 452]
 [455 480]]
[[414 378]
 [468 354]
 [514 458]
 [459 482]]
[[403 453]
 [428 369]
 [645 433]
 [620 517]]
[[405 449]
 [432 372]
 [612 434]
 [585 511]]
[[410 445]
 [437 373]
 [601 436]
 [574 508]]
[[411 450]
 [441 376]
 [594 438]
 [565 511]]
[[412 455]
 [442 377]
 [597 438]
 [567 515]]
[[423 458]
 [445 381]
 [641 436]
 [619 513]]
[[421 461]
 [449 381]
 [611 436]
 [584 516]]
[[430 455]
 [453 383]
 [615 435]
 [592 507]]
[[434 453]
 [459 381]
 [610 434]
 [585 506]]
[[442 461]
 [460 389]
 [640 434]
 [622 507]]
[[347  75]
 [373  29]
 [512 106]
 [487 152]]
[[345  78]
 [370  30]
 [514 105]
 [489 153]]
[[437 464]
 [454 394]
 [640 437]
 [624 507]]
[[343 127]
 [355  58]
 [507  84]
 [495 153]]
[[349  77]

[[296  95]
 [342  41]
 [484 162]
 [438 216]]
[[297  92]
 [375  38]
 [518 246]
 [440 300]]
[[295  91]
 [374  36]
 [520 248]
 [440 302]]
[[294  93]
 [343  38]
 [485 166]
 [435 221]]
[[296  91]
 [374  37]
 [518 247]
 [440 301]]
[[302 103]
 [347  47]
 [483 158]
 [438 214]]
[[302 103]
 [347  47]
 [483 158]
 [438 214]]
[[295  95]
 [342  40]
 [483 162]
 [435 217]]
[[303  98]
 [349  46]
 [482 161]
 [436 213]]
[[296  93]
 [343  40]
 [485 164]
 [438 217]]
[[303 101]
 [349  47]
 [484 163]
 [438 216]]
[[294  95]
 [342  40]
 [486 168]
 [438 222]]
[[303 104]
 [347  50]
 [482 160]
 [438 215]]
[[297  96]
 [342  43]
 [483 163]
 [438 216]]
[[295  96]
 [345  41]
 [488 172]
 [438 227]]
[[294  99]
 [340  43]
 [485 161]
 [439 217]]
[[294  96]
 [343  40]
 [487 168]
 [438 223]]
[[309  93]
 [384  46]
 [515 259]
 [440 305]]
[[296  97]
 [342  43]
 [484 165]
 [438 219]]
[[296  97]
 [342  43]
 [484 165]
 [438 219]]
[[295  98]
 [345  43]
 [485 171]
 [435 226]]
[[295  97]
 [340  43]
 [483 164]
 [438 217]]
[[304 101]

# با گرفتن تصویر بارکد را تشخیص میدهد وآنرا رمزگشایی میکندzbar از کتابخانه  decode  تابع 

 صاف شودprespective این تابع نسبت به چرخش بارکد مقاوم نیست و در صورت چرخش باید با تابع 

gradient = cv2.convertScaleAbs(sub) تبدیل اعداد شناور به داده های 8 بیتی

# دایره و بیضی

دایره

تابع زیر یک کانتور را بوسیله کوچکترین دایره ممکن تقریب میزند

# (Cx,Cy),r = cv2.minEnclosingCircle(points)

تابع زیر یک کانتور را بوسیله کوچکترین بیضی ممکن تقریب میزند

# (Cx,Cy),(MA,ma),angle = cv2.fitElipse(points)

(Cx,Cy)=مرکز بیضی

(MA,ma)=قطر بزرگ و کوچک بیضی

angle = زاویه چرخش بیضی

مثال :مقایسه انواع تقریب روی کانتور

In [2]:
import numpy as np
import cv2

image = cv2.imread("hand.jpg")
gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
blurred = cv2.GaussianBlur(gray,(3,3),0)
edged = cv2.Canny(blurred,30,180)

mode = cv2.RETR_EXTERNAL
method = cv2.CHAIN_APPROX_SIMPLE

cnts = cv2.findContours(edged.copy(),mode,method)[0]
cnt = max(cnts,key = cv2.contourArea)
p = cv2.arcLength(cnt,True)

##########چندضلعی غیر منتظم
approx = cv2.approxPolyDP(cnt,0.001*p,True)
image1 = image.copy()
cv2.polylines(image1,[approx],True,0,3)
cv2.imshow("Polygon",image1)
cv2.waitKey()
########چهارضلعی ثابت
(x,y,w,h) = cv2.boundingRect(cnt)
image2 = image.copy()
cv2.rectangle(image2,(x,y),(x+w,y+h),0,3)
cv2.imshow("Starting bounding Rect",image2)
cv2.waitKey()
#######چهارضلعی دورانی
rect = cv2.minAreaRect(cnt)
box = np.int0(cv2.boxPoints(rect))
image3 = image.copy()
cv2.polylines(image3,[box],True,0,3)
#cv2.rectangle(image3,box[0],box[3],0,3)
cv2.imshow("Rotated Rect",image3)
cv2.waitKey()
######دایره
(x,y),r = cv2.minEnclosingCircle(cnt)
center = (int(x),int(y))
r = int(r)
image4 = image.copy()
cv2.circle(image4,center,r,0,3)
cv2.imshow("Enclosing circle",image4)
cv2.waitKey()
##########بیضی
ellipse = cv2.fitEllipse(cnt)
image5 = image.copy()
cv2.ellipse(image5,ellipse,0,3)
cv2.imshow("Ellipse",image5)
cv2.waitKey(0)
cv2.destroyAllWindows()



C:\Users\Admin\AppData\Local\Temp\ipykernel_14096\3374948106.py:30: DeprecationWarning: `np.int0` is a deprecated alias for `np.intp`.  (Deprecated NumPy 1.24)
  box = np.int0(cv2.boxPoints(rect))


مثال:شمارش تعداد انگشتان دست بصورت بلادرنگ

In [6]:
import cv2
import numpy as np
from scipy.spatial import distance as dist

def hand_detect (image):
    global tex_hist,ker_1,ker_2
    result = []
    
    hsv = cv2.cvtColor(image,cv2.COLOR_BGR2HSV) 
    skin = cv2.calcBackProject([hsv],[0,1],tex_hist,[0,180,0,256],1)#ماسکی که کف دست را نشان میدهد
    cv2.filter2D(skin,-1,ker_1,skin)
    skin = cv2.GaussianBlur(skin,(9,9),0)
    skin = cv2.threshold(skin,250,255,cv2.THRESH_BINARY)[1]
    cv2.imshow("Skins detected",skin)
    
    
    for mode in ("palm","hand"):
        if(mode=="palm"):
            mask = cv2.erode(skin.copy(),ker_1,iterations=3)
            mask = cv2.dilate(mask.copy(),ker_1,iterations=4)
        
        elif(mode=="hand"):
            mask = skin.copy()
        
        _,labels = cv2.connectedComponents(mask,connectivity=8,ltype=cv2.CV_32S)#شماره گذاری اجسام داخل ماسک
        new_labels = []
        nzero_pix = []
       # print(np.unique(labels))
        for label in np.unique(labels):
            if (label==0):
                continue
            new_labels.append(label)
            labelMask = np.zeros(mask.shape,dtype = "uint8")
            labelMask[labels==label] = 255#نمایش هر جسم بصورت جدا درماسک جدید
            num_pix = cv2.countNonZero(labelMask)#شمارش تعداد پیکسل های هر جسم در ماسک جدید
            nzero_pix.append(num_pix)
        
        label = [new_labels for _,new_labels in sorted(zip(nzero_pix,new_labels),reverse=True)][0]#مرتب سازی شماره ها بر اساس تعداد پیکس هایشان
        final_mask = np.zeros(mask.shape,dtype="uint8")
        final_mask[labels == label] = 255
        final_mask = cv2.dilate(final_mask.copy(),ker_2,iterations=10)
        result.append(final_mask)
    return result

camera = cv2.VideoCapture(0)
font = cv2.FONT_HERSHEY_SIMPLEX
ker_1 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(11,11))
ker_2 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(5,1))

texture = cv2.imread("skin.png")
tex_hsv = cv2.cvtColor(texture,cv2.COLOR_BGR2HSV)
tex_hist = cv2.calcHist([tex_hsv],[0,1],None,[180,256],[0,180,0,256])
cv2.normalize(tex_hist,tex_hist,0,255,cv2.NORM_MINMAX)

while(True):
    _,frame = camera.read()
    frame = cv2.flip(frame,1)
    clone = frame.copy()
    (palm,hand) = hand_detect(frame)
    cv2.imshow("Palm",palm)
    cv2.imshow("Hand",hand)
    
    mode = cv2.RETR_EXTERNAL
    method = cv2.CHAIN_APPROX_SIMPLE
    palm_cnt = cv2.findContours(palm.copy(),mode,method)[0]
    palm_cnt = palm_cnt[0]
    
    (x,y),r = cv2.minEnclosingCircle(palm_cnt)
    center = (int(x),int(y))
    r = int(r*0.8)
    cv2.circle(clone,center,5,(0,0,255),-1)
    cv2.circle(clone,center,r,(0,255,0),3)
    
    center_down = (center[0],center[1]+int(r*0.5))
    cv2.circle(clone,center_down,5,(0,0,255),-1)
    
    hand_cnt = cv2.findContours(hand.copy(),mode,method)[0]
    hand_cnt = hand_cnt[0]
    
    p = cv2.arcLength(hand_cnt,True)
    approx = cv2.approxPolyDP(hand_cnt,0.02*p,True)
    cv2.polylines(clone,[approx],True,(255,255,255),3)
    points = []
    
    for pts in approx:
        pts = pts[0]
        d = dist.euclidean(pts,center)
        if ((d>r*1.6) and pts[1]<center_down[1]):
            points.append([pts])
            cv2.circle(clone,tuple(pts),5,(0,0,255),-1)
    cv2.putText(clone,"Number:{}".format(len(points)),(10,30),font,1,0,3)
    cv2.imshow("Frame",clone)
    if cv2.waitKey(1)==27:
        break
camera.release()
cv2.destroyAllWindows()
    
    
    
    

ValueError: too many values to unpack (expected 2)

محاسبه فاصله کانتور تا یک نقطه

# value = cv2.pointPolygonTest(contour,pts,measureDist)

pts , contoure = کانتور ونقطه ای که قرار است نسبت بهم سنجیده شوند

measureDist = (True،+بیرون کانتور-،داخل کانتور)یا فاصله آن دو محاسبه شود
(false,+بیرون کانتور1-،داخل کانتور1)این پارامتر بولی است که مشخص میکند موقعیت نسبت به کانتور سنجیده شود


value = اندازه فاصله یا موقعیت نقطه نسبت به کانتور

مثال:محاسبه فاصله اجسام تا جسم مرجع

In [ ]:
import cv2
import numpy as np
from scipy.spatial import distance as dist

font = cv2.FONT_HERSHEY_TRIPLEX
base_cm = 2.5 #قطر جسم مرجع بر حسب سانتی متر
pix_per_cm = 37.8 # تعداد پیکس در یک سانتی متر
base_pix = base_cm * pix_per_cm

def SortedContours(thresh):
    center = []
    sorted_cnts = []
    mode = cv2.RETR_EXTERNAL
    method = cv2.CHAIN_APPROX_SIMPLE
    cnts = cv2.findContours(thresh,mode,method)
    for c in cnts:
        if cv2.contourArea(c)>200:
            M = cv2.moments(c)
            cx = int(M["m10"]/M["m00"])
            center.append([cx,c])
    sorting = sorted(center,key = lambda center:center[0]) 
    for s in sorting:
        sorted_cnts.append((s[1]))
    return sorted_cnts


def Coefficient(contour):
    (x,y),(w,h),angle = cv2.minAreaRect(contour)
    coef = w/float(base_pix)
    return coef

image = cv2.imread

مقایسه کانتورها

# value = cv2.matchShapes(object1,object2,method,parameter)این تابع فقط برای مقایسه 2 کانتور به کار میرود

object1,object2 = ورودی های 1 و2 که باید تصویری خاکستری یا باینری یا کانتور باشد

method = CV_CONTOURS_MATCH_I1 , CV_CONTOURS_MATCH_I2 , CV_CONTOURS_MATCH_I3

parameter= این پارامتر قابل استفاده نمیباشد و برابر با صفر میگذاریم

value = میزان شباهت که به صورت عدد بازگردانده میشود



مثال:شناسایی اشکال هندسی به روش مقایسه ای